In [3]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import pandas as pd
import numpy as np
from nltk.stem.porter import PorterStemmer
import nltk
import gensim
from gensim import corpora

In [4]:
app = pd.read_pickle('app_cleaned.pickle')

In [5]:
app.columns

Index([u'category', u'current_rating', u'description', u'id',
       u'is_InAppPurcased', u'is_multilingual', u'is_multiplatform', u'name',
       u'new_version_desc', u'num_current_rating', u'num_overall_rating',
       u'overall_rating', u'price', u'publish_date', u'review1',
       u'review1_star', u'review2', u'review2_star', u'review3',
       u'review3_star', u'scrape_date', u'seller', u'size', u'update_date',
       u'url', u'version'],
      dtype='object')

Simply use current rating to define the quality of a app. If the current rating is no less than 4.0, it can be seen as a good app. If the current rating is no more than 2.5, it is a bad app.

In [6]:
good_app = app.loc[app['current_rating'] >=4.0]
bad_app = app.loc[app['current_rating'] <=2.5]
good_app = good_app.reset_index(drop=True)
bad_app = bad_app.reset_index(drop=True)


In [7]:
category = app['category']
cate_list = []
for i in category.unique():
    cate = i.lower()
    cate_list.append(cate)

Use star value of different reviews to filter comments. 

In [9]:
first_good= good_app.loc[good_app['review1_star']>=4].reset_index(drop=True)['review1']
second_good = good_app.loc[good_app['review2_star']>=4].reset_index(drop=True)['review2']
third_good = good_app.loc[good_app['review3_star']>=4].reset_index(drop=True)['review3']
first_bad = bad_app.loc[bad_app['review1_star']<=2.5].reset_index(drop=True)['review1']
second_bad = bad_app.loc[bad_app['review2_star']<=2.5].reset_index(drop=True)['review2']
third_bad = bad_app.loc[bad_app['review3_star']<=2.5].reset_index(drop=True)['review3']

In [10]:
good_rev = first_good.append(second_good)
all_good = good_rev.append(third_good)
bad_rev = first_bad.append(second_bad)
all_bad = bad_rev.append(third_bad)

<b>Cleaning and Preprocessing</b>

Data cleaning is absolutely crucial for generating a useful topic model. The steps below are common to most natural language processing methods:
   * Tokenizing: converting a document to its atomic elements.
   * Stopping: removing meaningless words.
   * Stemming: merging words that are equivalent in meaning.

Here we need to note that POS tag filter is more about the context of the features than frequencies of features. Topic Modelling tries to map out the recurring patterns of terms into topics. However, every term might not be equally important contextually. For example, POS tag IN contain terms such as – “within”, “upon”, “except”. “CD” contains – “one”,”two”, “hundred” etc. “MD” contains “may”, “must” etc. These terms are the supporting words of a language and can be removed by studying their post tags.

In [11]:
stop = set(stopwords.words('english')+[u'one',u'app',u'it',u'dont',u"i",u"'s","''","``",u'use',u'used',u'using',u'love',
                                      u'would',u'great',u'app.',u'like',u'lot']+ cate_list)
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

def stem(tokens,stemmer = PorterStemmer().stem):
    return [stemmer(w.lower()) for w in tokens if w not in stop]

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    tokenize = nltk.word_tokenize
    to_token = stem(tokenize(normalized))
    tags = nltk.pos_tag(to_token)
    dt_tags = [t for t in tags if t[1] in ["DT", "MD", "VBP","IN", "JJ","VB"]]
    for tag in dt_tags:
        normalized = " ".join(tok for tok in to_token if tok not in tag[0])
    return normalized

In [9]:
doc_clean_g1 = [clean(doc).split() for doc in first_good]
doc_clean_g2 = [clean(doc).split() for doc in second_good]
doc_clean_g3 = [clean(doc).split() for doc in third_good]
doc_clean_b1 = [clean(doc).split() for doc in first_bad]
doc_clean_b2 = [clean(doc).split() for doc in second_bad]
doc_clean_b3 = [clean(doc).split() for doc in third_bad]

In [12]:
doc_clean_good = [clean(doc).split() for doc in all_good]
doc_clean_bad = [clean(doc).split() for doc in all_bad]

<b>Preparing Document-Term Matrix</b>
* Convert a corpus into a document-term matrix. LDA model looks for repeating term patterns in the entire DT matrix

In [11]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index.
dictionary_g1 = corpora.Dictionary(doc_clean_g1)
dictionary_g2 = corpora.Dictionary(doc_clean_g2)
dictionary_g3 = corpora.Dictionary(doc_clean_g3)
dictionary_b1 = corpora.Dictionary(doc_clean_b1)
dictionary_b2 = corpora.Dictionary(doc_clean_b2)
dictionary_b3 = corpora.Dictionary(doc_clean_b3)

In [13]:
dictionary_good = corpora.Dictionary(doc_clean_good)
dictionary_bad = corpora.Dictionary(doc_clean_bad)

In [13]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix_g1 = [dictionary_g1.doc2bow(doc) for doc in doc_clean_g1]
doc_term_matrix_g2 = [dictionary_g2.doc2bow(doc) for doc in doc_clean_g2]
doc_term_matrix_g3 = [dictionary_g3.doc2bow(doc) for doc in doc_clean_g3]
doc_term_matrix_b1 = [dictionary_b1.doc2bow(doc) for doc in doc_clean_b1]
doc_term_matrix_b2 = [dictionary_b2.doc2bow(doc) for doc in doc_clean_b2]
doc_term_matrix_b3 = [dictionary_b3.doc2bow(doc) for doc in doc_clean_b3]

In [14]:
doc_term_matrix_good = [dictionary_good.doc2bow(doc) for doc in doc_clean_good]
doc_term_matrix_bad = [dictionary_bad.doc2bow(doc) for doc in doc_clean_bad]

<b>Running LDA Model (Batch Wise LDA)</b>
   * According to the [reference](https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/), in order to retrieve most important topic terms, a corpus can be divided into batches of fixed sizes. Running LDA multiple times on these batches will provide different results, however, the best topic terms will be the intersection of all batches. 


In [15]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

In [34]:
# Running and Trainign LDA model on the document term matrix.
ldamodel_g1 = Lda(doc_term_matrix_g1, num_topics=3, id2word = dictionary_g1, passes=50)
ldamodel_g2 = Lda(doc_term_matrix_g2, num_topics=3, id2word = dictionary_g2, passes=50)
ldamodel_g3 = Lda(doc_term_matrix_g3, num_topics=3, id2word = dictionary_g3, passes=50)
ldamodel_b1 = Lda(doc_term_matrix_b1, num_topics=3, id2word = dictionary_b1, passes=50)
ldamodel_b2 = Lda(doc_term_matrix_b2, num_topics=3, id2word = dictionary_b2, passes=50)
ldamodel_b3 = Lda(doc_term_matrix_b3, num_topics=3, id2word = dictionary_b3, passes=50)

<b>Examining the results</b>

In [35]:
print(ldamodel_g1.print_topics(num_topics=3, num_words=5))
print(ldamodel_g2.print_topics(num_topics=3, num_words=5))
print(ldamodel_g3.print_topics(num_topics=3, num_words=5))

[(0, u'0.011*year + 0.010*ive + 0.009*best + 0.008*im + 0.008*time'), (1, u'0.011*time + 0.007*want + 0.007*realli + 0.007*get + 0.006*also'), (2, u'0.022*get + 0.011*check + 0.011*need + 0.011*forecast + 0.009*hope')]
[(0, u'0.011*get + 0.010*game + 0.009*time + 0.008*realli + 0.007*help'), (1, u'0.010*time + 0.008*easi + 0.008*make + 0.007*work + 0.007*get'), (2, u'0.006*time + 0.006*year + 0.006*day + 0.006*work + 0.005*go')]
[(0, u'0.014*free + 0.013*call + 0.011*phone + 0.010*make + 0.010*work'), (1, u'0.033*screen + 0.030*know + 0.019*go + 0.018*everi + 0.017*see'), (2, u'0.022*right + 0.018*also + 0.017*calcul + 0.013*away + 0.013*need')]


Each generated topic is separated by a comma. Within each topic are the five most probable words to appear in that topic. The best topic terms will be the intersection of all three batches. Some things to think about, for the good app, the comments have common features like: 
  1. It's free have some good features that satisfy customers' demand.
  2. It has many good information and details, and customers are comfortable at vision, like screen.
  3. The speed is awesome and save some time.
  4. It provids some help when customers using it.


In [36]:
print(ldamodel_b1.print_topics(num_topics=3, num_words=5))
print(ldamodel_b2.print_topics(num_topics=3, num_words=5))
print(ldamodel_b3.print_topics(num_topics=3, num_words=5))

[(0, u'0.010*time + 0.010*get + 0.009*work + 0.009*even + 0.007*tri'), (1, u'0.011*time + 0.007*updat + 0.007*work + 0.007*get + 0.006*im'), (2, u'0.011*time + 0.007*tri + 0.006*phone + 0.006*go + 0.006*get')]
[(0, u'0.012*get + 0.008*work + 0.008*tri + 0.008*time + 0.007*im'), (1, u'0.014*time + 0.012*get + 0.009*work + 0.007*need + 0.007*even'), (2, u'0.011*time + 0.009*work + 0.009*version + 0.007*tri + 0.007*updat')]
[(0, u'0.011*updat + 0.010*back + 0.009*work + 0.007*read + 0.006*tri'), (1, u'0.008*get + 0.008*work + 0.007*ad + 0.006*time + 0.006*doesnt'), (2, u'0.018*time + 0.011*get + 0.009*even + 0.008*tri + 0.007*work')]


For the bad apps, from the result, we can see most topics include the word "time". We can refer that customers are not satisfied for the using fluency of these apps. And for the updated version of these apps, they doesn't work sometimes, maybe because flashing back. Meanwhile, compared with the last version, these updated apps maybe designed not that good. 


<b>Running LDA Model (For the whole documents)</b>

In [16]:
ldamodel_good = Lda(doc_term_matrix_good, num_topics=10, id2word = dictionary_good, passes=20)
ldamodel_bad = Lda(doc_term_matrix_bad, num_topics=10, id2word = dictionary_bad, passes=20)

In [32]:
print(ldamodel_good.print_topics(num_topics=5, num_words=3))
print(ldamodel_bad.print_topics(num_topics=5, num_words=3))

[(5, u'0.040*open + 0.029*see + 0.027*screen'), (2, u'0.013*time + 0.011*im + 0.011*get'), (9, u'0.030*accur + 0.027*also + 0.025*time'), (7, u'0.026*day + 0.020*everi + 0.014*help'), (3, u'0.021*inform + 0.020*locat + 0.020*map')]
[(6, u'0.013*time + 0.011*work + 0.010*even'), (9, u'0.014*work + 0.013*time + 0.009*doesnt'), (0, u'0.013*get + 0.012*work + 0.012*time'), (5, u'0.012*page + 0.011*time + 0.010*get'), (2, u'0.016*time + 0.009*need + 0.007*show')]


In [17]:
import pyLDAvis
import pyLDAvis.gensim

In [19]:
pyLDAvis.enable_notebook()
good_rev = pyLDAvis.gensim.prepare(ldamodel_good, doc_term_matrix_good, dictionary_good)
bad_rev = pyLDAvis.gensim.prepare(ldamodel_bad, doc_term_matrix_bad, dictionary_bad)

In [22]:
good_rev

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
8      25.459693        1       1  0.186887  0.009498
3      24.094634        1       2  0.195694 -0.022315
4      20.276995        1       3  0.175441  0.019071
0       8.583486        1       4  0.004529 -0.004001
5       4.530929        1       5  0.044791  0.174326
1       4.032727        1       6 -0.146969 -0.042599
7       3.759459        1       7 -0.011402 -0.150277
6       3.655921        1       8 -0.036049 -0.160926
2       3.146087        1       9 -0.167219  0.270436
9       2.460070        1      10 -0.245703 -0.093212, topic_info=      Category         Freq     Term        Total  loglift  logprob
term                                                               
8595   Default   544.000000   inform   544.000000  30.0000  30.0000
1659   Default   780.000000      see   780.000000  29.0000  29.0000
4846   Default   878.000000     need   878.000000  28.0000  28.0000
5662   Default   892.000000       im   892.000000  27.0000  27.0000
658    Default  1415.000000     time  1415.000000  26.0000  26.0000
7896   Default   519.000000    phone   519.000000  25.0000  25.0000
1582   Default  1117.000000     also  1117.000000  24.0000  24.0000
6206   Default   409.000000     show   409.000000  23.0000  23.0000
4363   Default   745.000000     look   745.000000  22.0000  22.0000
4446   Default   624.000000     game   624.000000  21.0000  21.0000
6225   Default  1645.000000      get  1645.000000  20.0000  20.0000
7019   Default   738.000000      app   738.000000  19.0000  19.0000
2773   Default  1069.000000     work  1069.000000  18.0000  18.0000
5248   Default   699.000000     even   699.000000  17.0000  17.0000
394    Default   607.000000     best   607.000000  16.0000  16.0000
2723   Default   219.000000   calcul   219.000000  15.0000  15.0000
8001   Default   485.000000     free   485.000000  14.0000  14.0000
7047   Default   464.000000     back   464.000000  13.0000  13.0000
8828   Default   464.000000   review   464.000000  12.0000  12.0000
4098   Default   868.000000     help   868.000000  11.0000  11.0000
6567   Default   570.000000      abl   570.000000  10.0000  10.0000
115    Default   714.000000     want   714.000000   9.0000   9.0000
6845   Default   602.000000     find   602.000000   8.0000   8.0000
5332   Default   534.000000     give   534.000000   7.0000   7.0000
6326   Default   310.000000      pay   310.000000   6.0000   6.0000
5039   Default   655.000000     well   655.000000   5.0000   5.0000
548    Default   531.000000    right   531.000000   4.0000   4.0000
5092   Default   219.000000     home   219.000000   3.0000   3.0000
896    Default   227.000000     open   227.000000   2.0000   2.0000
4576   Default   868.000000       go   868.000000   1.0000   1.0000
...        ...          ...      ...          ...      ...      ...
10395  Topic10    38.464002  whether    60.198076   3.2571  -4.6539
6224   Topic10    35.875030   variou    57.211963   3.2383  -4.7236
896    Topic10   120.165343     open   227.095768   3.0685  -3.5147
8595   Topic10   203.114291   inform   544.731801   2.7185  -2.9898
549    Topic10    67.612327      old   155.757229   2.8705  -4.0898
7862   Topic10    31.521767     path    65.937025   2.9670  -4.8529
4251   Topic10    38.089134    clear    84.555984   2.9075  -4.6637
2932   Topic10    75.607045   provid   216.409537   2.6534  -3.9781
722    Topic10    66.410996  current   183.693633   2.6876  -4.1077
4734   Topic10    36.682838    handi    90.614157   2.8007  -4.7013
9137   Topic10    35.311648   follow    88.295045   2.7885  -4.7394
4165   Topic10    34.191725     base    91.756487   2.7178  -4.7716
4014   Topic10    54.312951   famili   218.935244   2.3110  -4.3088
3581   Topic10    65.261302   friend   323.210755   2.1051  -4.1252
10123  Topic10    41.012581    locat   135.448265   2.5103  -4.5897
4846   Topic10    93.681861     need   878.08

In [21]:
bad_rev

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      24.086025        1       1 -0.054506  0.014774
5      15.259739        1       2 -0.056139  0.003142
0      11.537455        1       3 -0.045449 -0.001813
3      11.403085        1       4 -0.057969 -0.009376
8       9.653466        1       5 -0.041715 -0.008723
7       9.557511        1       6 -0.026137  0.007692
9       6.429321        1       7  0.081664  0.153531
4       5.802608        1       8  0.001341 -0.021521
2       4.591876        1       9  0.024879 -0.078103
6       1.678915        1      10  0.174031 -0.059605, topic_info=     Category        Freq      Term       Total  loglift  logprob
term                                                             
186   Default   92.000000      give   92.000000  30.0000  30.0000
2220  Default  126.000000        ad  126.000000  29.0000  29.0000
2903  Default  109.000000      find  109.000000  28.0000  28.0000
172   Default  337.000000      work  337.000000  27.0000  27.0000
1779  Default  354.000000       get  354.000000  26.0000  26.0000
76    Default  130.000000       new  130.000000  25.0000  25.0000
4023  Default   66.000000     locat   66.000000  24.0000  24.0000
597   Default   61.000000     watch   61.000000  23.0000  23.0000
3947  Default  183.000000      need  183.000000  22.0000  22.0000
3630  Default   77.000000    websit   77.000000  21.0000  21.0000
3028  Default   67.000000      load   67.000000  20.0000  20.0000
1184  Default   42.000000      item   42.000000  19.0000  19.0000
2400  Default  137.000000   version  137.000000  18.0000  18.0000
2855  Default  131.000000      also  131.000000  17.0000  17.0000
3783  Default  107.000000       day  107.000000  16.0000  16.0000
1015  Default  104.000000       way  104.000000  15.0000  15.0000
2359  Default   56.000000      list   56.000000  14.0000  14.0000
3575  Default   95.000000      sinc   95.000000  13.0000  13.0000
3654  Default  108.000000   account  108.000000  12.0000  12.0000
88    Default  108.000000     never  108.000000  11.0000  11.0000
1542  Default   59.000000      card   59.000000  10.0000  10.0000
2317  Default   35.000000        tv   35.000000   9.0000   9.0000
783   Default  186.000000      cant  186.000000   8.0000   8.0000
2791  Default   83.000000     store   83.000000   7.0000   7.0000
219   Default  245.000000     updat  245.000000   6.0000   6.0000
2670  Default   32.000000     found   32.000000   5.0000   5.0000
2179  Default   73.000000     start   73.000000   4.0000   4.0000
697   Default   46.000000     login   46.000000   3.0000   3.0000
193   Default  157.000000      want  157.000000   2.0000   2.0000
4347  Default   54.000000  password   54.000000   1.0000   1.0000
...       ...         ...       ...         ...      ...      ...
13    Topic10    2.400138     accur   13.007310   2.3970  -5.6086
3034  Topic10    1.324420      loan    4.717833   2.8166  -6.2031
2567  Topic10    1.791198   catalog    8.197717   2.5661  -5.9012
2072  Topic10    1.324331   realtim    4.857483   2.7874  -6.2032
2454  Topic10    1.308757  suddenli    4.841958   2.7788  -6.2150
186   Topic10    5.915702      give   92.885719   1.3333  -4.7065
1727  Topic10    2.488685     drive   18.805659   2.0646  -5.5723
2545  Topic10    2.785946    rather   24.414981   1.9164  -5.4595
2036  Topic10    2.585620     littl   22.019155   1.9451  -5.5341
2670  Topic10    3.042292     found   32.537133   1.7173  -5.3715
2359  Topic10    3.499272      list   56.157550   1.3114  -5.2315
2903  Topic10    4.392793      find  109.906887   0.8674  -5.0041
260   Topic10    2.103994       fan   19.233683   1.8742  -5.7403
2179  Topic10    3.077838     start   73.991250   0.9073  -5.3599
2639  Topic10    3.184027      good   85.341452   0.7985  -5.3259
3028  Topic10    2.960479      load   67.744920   0.9566  -5.3987
1015  Topic10    3.321782       way  104.150311   0.6417  -5.2836
3575  Top